In [1]:
import os

In [2]:
%pwd

'/home/joshua/Customer-Segmentation-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/joshua/Customer-Segmentation-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    transformed_data_path: Path
    clustered_data_path: Path
    model_save_path: Path
    test_size: float
    random_state: int
    n_estimators: int
    max_depth: int

In [6]:
# configuration_manager.py
from customer_segmentation.constants import *
from customer_segmentation.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.model_trainer.root_dir])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_trainer

        return ModelTrainerConfig(
            root_dir=config.root_dir,
            transformed_data_path=config.transformed_data_path,
            clustered_data_path=config.clustered_data_path,
            model_save_path=config.model_save_path,
            test_size=params.test_size,
            random_state=params.random_state,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth
        )

In [7]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        # Load transformed data
        data = pd.read_csv(self.config.transformed_data_path)

        # KMeans Clustering
        kmeans = KMeans(n_clusters=4, random_state=self.config.random_state, n_init=10)
        data['Cluster'] = kmeans.fit_predict(data.drop('User_ID', axis=1))
        data[['User_ID', 'Cluster']].to_csv(self.config.clustered_data_path, index=False)

        # Prepare for Random Forest
        X = data.drop(['User_ID', 'Cluster'], axis=1)
        y = data['Cluster']
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=self.config.test_size, random_state=self.config.random_state
        )

        rf = RandomForestClassifier(
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            random_state=self.config.random_state
        )
        rf.fit(X_train, y_train)
        y_pred = rf.predict(X_test)
        print(classification_report(y_test, y_pred))

        # Save the trained model
        joblib.dump(rf, self.config.model_save_path)

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-02-23 14:10:15,673: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-23 14:10:15,680: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 14:10:15,684: INFO: common: created directory at: artifacts/model_trainer]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       357
           1       1.00      0.99      0.99       256
           2       0.99      0.98      0.99       360
           3       0.98      0.98      0.98       206

    accuracy                           0.99      1179
   macro avg       0.99      0.99      0.99      1179
weighted avg       0.99      0.99      0.99      1179

